In [4]:
import requests
from bs4 import BeautifulSoup

In [6]:
pages = [
    'https://www.pgm-my.org/malaysianjournalofgenetics/december-2022/',
    'https://www.pgm-my.org/malaysianjournalofgenetics/december-2021/'
]

In [21]:
urls = []
for p in pages:
    r = requests.get(p)
    soup = BeautifulSoup(r.content)
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if 'drive.google.com' in a_.get('href') or a_.get('href').endswith('.pdf')]
    urls.extend(a)

In [23]:
urls = sorted(list(set(urls)))

In [22]:
!mkdir pgm-my.org

In [25]:
import os
import gdown
from tqdm import tqdm

for i in tqdm(range(len(urls))):
    filename = os.path.join('pgm-my.org', f'{i}.pdf')
    if os.path.exists(filename):
        continue
    
    u = urls[i]
    gdown.download(output = filename, id = u.split('/d/')[1].split('/')[0])

  0%|                                                    | 0/12 [00:00<?, ?it/s]Downloading...
From: https://drive.google.com/uc?id=18VfD_4s4KnO60aa7ZocCkkz5eKqr1IYx
To: /home/husein/ssd3/crawl-my-websites/myjms.mohe.gov.my/pgm-my.org/0.pdf

100%|████████████████████████████████████████| 517k/517k [00:00<00:00, 7.13MB/s]
  8%|███▋                                        | 1/12 [00:02<00:28,  2.63s/it]Downloading...
From: https://drive.google.com/uc?id=1CCNuRoNEUXmsKcYXjXcSyO5IqUsn5wqd
To: /home/husein/ssd3/crawl-my-websites/myjms.mohe.gov.my/pgm-my.org/1.pdf

100%|████████████████████████████████████████| 404k/404k [00:00<00:00, 6.15MB/s]
 17%|███████▎                                    | 2/12 [00:05<00:25,  2.57s/it]Downloading...
From: https://drive.google.com/uc?id=1FiOfiCvdiK41VSVvWUwXuSQaiH_tyfPE
To: /home/husein/ssd3/crawl-my-websites/myjms.mohe.gov.my/pgm-my.org/2.pdf

100%|████████████████████████████████████████| 356k/356k [00:00<00:00, 5.75MB/s]
 25%|███████████               

In [31]:
from glob import glob
from tika import parser
import json

TIKA_HOST = 'http://localhost:9998'

In [32]:
files = glob('pgm-my.org/*.pdf')

with open('pgm-my.org.jsonl', 'w') as fopen:
    for f in files:
        raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
        body = BeautifulSoup(raw_xml['content']).find('body')
        t = '\n'.join(body.findAll(text=True))
        fopen.write(f'{json.dumps(t)}\n')